In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets( 'tensorflow/models', one_hot=True )

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tensorflow/models/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tensorflow/models/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting tensorflow/models/t10k-images-idx3-ubyte.gz
Extracting tensorflow/models/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
num_epochs = 5
learning_rate = 0.01
num_input = 784
num_classes = 10

In [5]:
with tf.variable_scope('input'):
    x = tf.placeholder(dtype=tf.float32,shape=[None,784])
    x_op = tf.reshape(x, shape=[-1, 28, 28, 1])

In [6]:
with tf.variable_scope('cnn_layer_1'):
    ## weights = self.add_weights([kernel, kernel, input_shape, output_shape])
    weights_1 = tf.Variable(tf.random_normal([5,5,1,32]))
    ## biases = self.add_biases([output_shape])
    biases_1 = tf.Variable(tf.random_normal([32]))
    ## stride=[image_jump,row_jump,column_jump,color_jump]=[1,1,1,1]
    stride = [1,1,1,1]
    ## layer = tf.nn.conv2d(layer, weights, strides=stride, padding='SAME') + biases  
    conv1 = tf.add(tf.nn.conv2d(x_op,weights_1,strides=stride, padding='SAME'),biases_1)
    conv1 = tf.nn.relu(conv1)

In [7]:
with tf.variable_scope('max_pool_layer_1'):
    #kernel=[image_jump,rows,columns,depth]
    kernel = [1, 2, 2, 1]
    #stride=[image_jump,row_jump,column_jump,color_jump]=[1,2,2,1] mostly
    stride = [1, 2, 2, 1]
    max_pool1 = tf.nn.max_pool(conv1, ksize=kernel, strides=stride, padding='SAME')

In [8]:
with tf.variable_scope('cnn_layer_2'):
    ## weights = self.add_weights([kernel, kernel, input_shape, output_shape])
    weights_2 = tf.Variable(tf.random_normal([5,5,32,64]))
    ## biases = self.add_biases([output_shape])
    biases_2 = tf.Variable(tf.random_normal([64]))
    ## stride=[image_jump,row_jump,column_jump,color_jump]=[1,1,1,1]
    stride = [1,1,1,1]
    ## layer = tf.nn.conv2d(layer, weights, strides=stride, padding='SAME') + biases  
    conv2 = tf.add(tf.nn.conv2d(max_pool1,weights_2,strides=stride, padding='SAME'),biases_2)
    conv2 = tf.nn.relu(conv2)

In [9]:
with tf.variable_scope('max_pool_layer_2'):
    #kernel=[image_jump,rows,columns,depth]
    kernel = [1, 2, 2, 1]
    #stride=[image_jump,row_jump,column_jump,color_jump]=[1,2,2,1] mostly
    stride = [1, 2, 2, 1]
    max_pool2 = tf.nn.max_pool(conv2, ksize=kernel, strides=stride, padding='SAME')

In [10]:
with tf.variable_scope('dense_layer_1'):
    max_pool2_flat = tf.reshape(max_pool2, [-1, 7 * 7 * 64])
    weights_3 = tf.Variable(tf.random_normal(shape=[7*7*64,1024]))
    biases_3 = tf.Variable(tf.random_normal(shape=[1024]))
    dense_1_output = tf.add(tf.matmul(max_pool2_flat,weights_3),biases_3)
    dense_1_output = tf.nn.relu(dense_1_output)

In [11]:
with tf.variable_scope('output'):
    weights_out = tf.Variable(tf.random_normal(shape=[1024,num_classes]))
    biases_out = tf.Variable(tf.random_normal(shape=[num_classes]))
    output = tf.add(tf.matmul(dense_1_output,weights_out),biases_out)

In [12]:
with tf.variable_scope('cost'):
    y = tf.placeholder(dtype=tf.float32,shape=[None,10])
    cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output,labels=y)
    cost = tf.reduce_mean(cost)

In [13]:
with tf.variable_scope('train'):
    opt = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [14]:
train_writer = tf.summary.FileWriter('./logs/training_cnn')
train_writer.add_graph( tf.get_default_graph() )

In [15]:
with tf.variable_scope('summary'):
    summary = tf.summary.scalar('cost',cost)

In [16]:
saver = tf.train.Saver()

In [18]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#     last_ckpt = tf.train.latest_checkpoint('./cnn_weights')
#     saver.restore(sess,last_ckpt)
    for epoch in range(num_epochs):
        print('Epoch: '+str(epoch+1))
        for batch in range(100):
            data_x,data_y = mnist.train.next_batch(32)
            sess.run(opt,feed_dict={
                x:data_x,
                y:data_y
            })
            data_x,data_y = mnist.train.next_batch(256)
            training_loss,train_summary = sess.run([cost,summary],feed_dict = {
                x:data_x,
                y:data_y
            })
            train_writer.add_summary(train_summary,epoch)
            test_loss=0.0
            for _ in range(50):
                data_x,data_y = mnist.test.next_batch(32)
                test_loss += sess.run(cost, feed_dict = {
                    x:data_x,
                    y:data_y
                })
            test_loss = test_loss/50
            if batch%10 == 0:
                print('Batch: '+str(batch))
                print ( 'Training Loss is ' + str(training_loss) + '\nTesting loss is ' + str(test_loss) + '\n')
                saver.save(sess, './cnn_weights/epoch_' + str(epoch + 1) + '.ckpt')
                print (" Model Saved \n ")
                print (" ----------- \n")

Epoch: 1
Batch: 0
Training Loss is 128005.72
Testing loss is 119714.7615625

 Model Saved 
 
 ----------- 

Batch: 10
Training Loss is 73201.39
Testing loss is 72812.506640625

 Model Saved 
 
 ----------- 

Batch: 20
Training Loss is 7264.216
Testing loss is 7088.743032226563

 Model Saved 
 
 ----------- 

Batch: 30
Training Loss is 1879.6907
Testing loss is 1709.1532556152345

 Model Saved 
 
 ----------- 

Batch: 40
Training Loss is 671.91595
Testing loss is 965.6611810302734

 Model Saved 
 
 ----------- 

Batch: 50
Training Loss is 743.83984
Testing loss is 610.9390895080567

 Model Saved 
 
 ----------- 

Batch: 60
Training Loss is 620.6316
Testing loss is 581.1547503662109

 Model Saved 
 
 ----------- 

Batch: 70
Training Loss is 447.62112
Testing loss is 527.5645080566406

 Model Saved 
 
 ----------- 

Batch: 80
Training Loss is 391.65933
Testing loss is 521.1587117004394

 Model Saved 
 
 ----------- 

Batch: 90
Training Loss is 506.84586
Testing loss is 451.2556408691406

